# ***Let's do RAG***

In [7]:
import os
import requests
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from IPython.display import Markdown
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv("CLARIN_TOKEN")
CLARIN_CHAT_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
MODEL_ID = "llama3.1"
PATH_TO_DB = "../db/chroma"
RAG_PROMPT = "You are an advanced AI assistant using retrieval-augmented generation to provide detailed and accurate responses. \
Use the following pieces of retrieved context from Andrew Huberman's teachings to answer the question. \
If you don't know the answer, say that you don't know.\n\n"

In [3]:
def initialize_vector_store():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    chroma_client = chromadb.PersistentClient(path=PATH_TO_DB)
    vector_store = Chroma(
        embedding_function=embeddings,
        client=chroma_client,
        collection_name="huberman_lab"
    )
    return vector_store


def get_top_docs(vector_store, query, k=6):
    return vector_store.similarity_search(query, k=k)


def get_llm_response(question):
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL_ID,
        "messages": [
            {
                "role": "user",
                "content": question
            }
        ]
    }
    response = requests.post(CLARIN_CHAT_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print("Response content:", response.text)
        return None

#### ***RAG response***

In [ ]:
vector_store = initialize_vector_store()

In [11]:
def get_rag_response(user_input):
    retrieved_docs = get_top_docs(vector_store, user_input)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    query = f"{RAG_PROMPT}\nRetrieved information:\n {context}\n\nQuestion: {user_input}"
    response_text = get_llm_response(query)
    return response_text['choices'][0]['message']['content']

user_input = "What is the best way to improve sleep quality?"
response = get_rag_response(user_input)
display(Markdown(response))

According to Andrew Huberman's teachings, the best way to improve sleep quality involves a multi-step approach, starting with behavioral tools:

1. Establishing a consistent daily routine, including exposure to sunlight in the afternoon.
2. Avoiding caffeine for 8-10 hours before bedtime.
3. Not consuming alcohol before bed.
4. Cooling down the bedroom.
5. Dimming the lights.

In addition to these behavioral tools, exercise can also be beneficial in improving sleep quality, specifically increasing slow-wave stability during slow-wave (deep) sleep.

#### ***NO RAG***

In [12]:
response = get_llm_response(user_input)['choices'][0]['message']['content']
display(Markdown(response))

Improving sleep quality can be achieved through a combination of lifestyle changes, habits, and environment modifications. Here are some evidence-based tips to help you enhance your sleep quality:

1. **Establish a consistent sleep schedule**: Go to bed and wake up at the same time every day, including weekends, to regulate your body's internal clock.
2. **Create a sleep-conducive environment**:
	* Make your bedroom dark, quiet, and cool (around 60-67°F/15-19°C).
	* Invest in a comfortable mattress and pillows.
	* Consider using blackout curtains, earplugs, or a white noise machine if necessary.
3. **Develop a relaxing bedtime routine**: Wind down with activities like reading, meditation, deep breathing exercises, or yoga before bed.
4. **Limit exposure to screens before bed**: Avoid using electronic devices for at least an hour before bedtime, as the blue light emitted from screens can suppress melatonin production.
5. **Avoid stimulating activities before bedtime**:
	* Avoid intense exercise, caffeine, nicotine, and heavy meals within 2-3 hours of bedtime.
	* Try to manage stress with relaxation techniques or journaling instead of engaging in stimulating conversations or watching exciting movies.
6. **Get regular physical activity**: Regular exercise can help improve sleep quality, but avoid vigorous exercise within a few hours of bedtime.
7. **Try progressive muscle relaxation**: This technique involves tensing and relaxing different muscle groups to release tension and promote relaxation.
8. **Limit naps**: If you need to nap, keep it short (under 30 minutes) and avoid napping close to bedtime.
9. **Consider cognitive behavioral therapy for insomnia (CBT-I)**: If you struggle with chronic insomnia, CBT-I may help address underlying issues and develop healthier sleep habits.
10. **Consult a healthcare professional**: If you continue to experience poor sleep quality or excessive daytime fatigue, consult with a healthcare expert to rule out any underlying medical conditions.

Remember that individual sleep needs vary, so experiment with these suggestions and find what works best for you.

References:

* National Sleep Foundation. (n.d.). Improve Your Sleep. Retrieved <https://www.sleepfoundation.org/articles/tips-to-improve-sleep>
* Harvard Health Publishing. (2019). Sleep Better, Live Better. Retrieved <https://www.health.harvard.edu/topics/sleep-better-live-better>

Additional recommendations based on scientific studies:

* Exposure to natural light: Aim for morning sunlight to boost alertness and help regulate your circadian rhythms (Khalsa et al., 2003)
* Aromatherapy: Incorporate calming scents like lavender, vanilla, or chamomile into your bedroom to promote relaxation (Goel & Etwaroo, 2020)

References:

Goel, N., & Etwaroo, M. E. (2020). Impact of aromatherapy on sleep quality in adults. Journal of Alternative and Complementary Medicine, 26(3), 236–244.

Khalsa, S. B. S., Jewett, M. E., Cajochen, C., & Czeisler, C. A. (2003). A phase response curve for the human circadian pacemaker. Physical Review Physiology, 82(12), 1120–1131.

#### **comment**: *Llama hallucinates like crazy.*

#### ***Comparison RAG VS no-RAG***

In [15]:
question = "Who is Andrew Huberman?"

rag_response = get_rag_response(question)
no_rag_response = get_llm_response(question)['choices'][0]['message']['content']

comparison_table = f"""
# RAG Response
{rag_response}

# No RAG Response
{no_rag_response}
"""

display(Markdown(comparison_table))


# RAG Response
Based on the provided information, Andrew Huberman is a professor of neurobiology and ophthalmology at Stanford School of Medicine. He teaches neuroscience and neuroscience-related themes, has a presence on social media, and hosts the Huberman Lab Podcast where he discusses science and science-based tools for everyday life, aiming to bring zero-cost information to the general public.

# No RAG Response
Andrew Huberman is an American neuroscientist and Professor of Neurobiology, Ophthalmology, and Psychiatry at the Stanford University School of Medicine. He is also a popular podcaster and social media personality.

Huberman received his Bachelor's degree in Biology and Psychology from the University of California, Davis, and his Ph.D. in Neuroscience from the University of California, Davis. He then completed a postdoctoral fellowship at the University of California, San Diego.

Huberman's research focuses on the neural circuits that govern visual perception, sleep, and emotional regulation. His work has been published in several high-impact scientific journals, including Nature, Science, and Neuron.

In addition to his academic work, Huberman is known for his ability to explain complex scientific concepts in simple terms. He has gained a large following on social media platforms, such as Instagram and Twitter, where he shares insights on neuroscience, brain development, and overall well-being.

He hosts a podcast called the "Huberman Lab Podcast" where he discusses science and health-related topics with various guests. The podcast covers a wide range of subjects, including neuroscience, psychology, physiology, and more.

Overall, Andrew Huberman is a prominent figure in the worlds of both academia and popular science, using his knowledge to educate people about the human brain and body.


## ***RAG with MMR***

In [37]:
def initialize_vector_store():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    chroma_client = chromadb.PersistentClient(path=PATH_TO_DB)
    vector_store = Chroma(
        embedding_function=embeddings,
        client=chroma_client,
        collection_name="huberman_lab"
    )
    return vector_store


def get_top_docs_mmr(vector_store, query, k=6, fetch_k=20, lambda_mult=0.5):
    return vector_store.max_marginal_relevance_search(
        query,
        k=k,
        fetch_k=fetch_k,
        lambda_mult=lambda_mult
    )

def get_top_docs_cosine(vector_store, query, k=6):
    return vector_store.similarity_search(query, k=k)

def get_llm_response(question, temperature=0.5):
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL_ID,
        "messages": [
            {
                "role": "user",
                "content": question,
            }
        ],
        "temperature": temperature
    }
    response = requests.post(CLARIN_CHAT_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print("Response content:", response.text)
        return None
    
def get_rag_response(user_input, temperature=0.1, similarity_search=get_top_docs_cosine):
    retrieved_docs = similarity_search(vector_store, user_input)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    query = f"{RAG_PROMPT}\nRetrieved information:\n {context}\n\nQuestion: {user_input}"
    response_text = get_llm_response(query, temperature)
    return response_text['choices'][0]['message']['content']

## ***RAG with Cosine Similarity vs MMR***

**question:** "who is Andrew Huberman?"

In [40]:
question = "who is Andrew Huberman?"
response = get_rag_response(question, temperature=0.3, similarity_search=get_top_docs_cosine)
display(Markdown(f"# Cosine similarity search"))
display(Markdown(f"<div style='font-size: 12pt'>{response}</div>"))
display(Markdown(f"--------------------------------"))
response = get_rag_response(question, temperature=0.3, similarity_search=get_top_docs_mmr)
display(Markdown(f"# MMR similarity search"))
display(Markdown(f"<div style='font-size: 12pt'>{response}</div>"))

# Cosine similarity search

<div style='font-size: 12pt'>Based on the provided context, Andrew Huberman is a professor of neurobiology and ophthalmology at Stanford School of Medicine. He hosts the Huberman Lab Podcast where he discusses science and science-based tools for everyday life, aiming to make this information available to the general public at no cost.</div>

--------------------------------

# MMR similarity search

<div style='font-size: 12pt'>Based on the provided context, Andrew Huberman is a professor of neurobiology and ophthalmology at Stanford School of Medicine. He hosts "The Huberman Lab Podcast" where he discusses science and science-based tools for everyday life, aiming to provide zero-cost information to the general public.</div>

## **Temperature**
**question:** "What is the best way to improve sleep quality?"

In [41]:
question = "What is the best way to improve sleep quality?"

for i in range(5):
    response = get_rag_response(question, temperature=0.5)
    display(Markdown(f"<div style='font-size: 12pt'>{response}</div>"))
    display(Markdown(f"--------------------------------"))

<div style='font-size: 12pt'>According to Andrew Huberman, the best approach to improving sleep quality involves a multi-step process. Firstly, he emphasizes the importance of establishing good "behavioral tools" which include having a consistent morning routine, getting sunlight exposure in the afternoon, avoiding caffeine and alcohol close to bedtime, cooling down the room, and dimming the lights. 

Additionally, exercise has been found to significantly improve the quality of slow-wave sleep by increasing slow-wave stability. He also mentions that nutrition can play a role in optimizing sleep, although this should be considered secondary to behavioral changes.

It's worth noting that while sleep trackers can provide some insights into sleep patterns, they are not perfectly accurate and should not be solely relied upon.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman's teachings, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some key takeaways:

1. **Behavioral Tools**: Establish a consistent morning and afternoon routine, including exposure to sunlight in the afternoon. Avoid caffeine and alcohol in the hours leading up to bedtime. Create a sleep-conducive environment by cooling down the room and dimming the lights.
2. **Nutrition**: While specific dietary recommendations are not mentioned in the provided text, Andrew Huberman suggests considering nutrition as a secondary approach to improving sleep quality.
3. **Exercise**: Regular exercise can help improve slow-wave sleep quality by increasing slow-wave stability. This is supported by research papers referenced by Andrew Huberman.

By incorporating these strategies, individuals can optimize their sleep quality, which is essential for overall well-being, energy levels, and mood.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some specific tips mentioned:

1. **Establish a consistent morning and afternoon routine**: This helps regulate the body's internal clock.
2. **Get sunlight exposure in the afternoon**: Natural light exposure helps regulate the circadian rhythm.
3. **Avoid caffeine and alcohol**: Both can disrupt sleep patterns; avoid consuming them in the 8-10 hours leading up to bedtime.
4. **Cool down the room**: A cooler environment promotes better sleep.
5. **Dim the lights**: Darkness helps signal to the brain that it's time to sleep.

Additionally, exercise has been shown to improve slow-wave sleep stability, which is a key aspect of deep sleep. However, it's essential to note that individual results may vary, and what works best for each person depends on their unique needs and preferences.

It's also important to remember that commercial sleep trackers are not always accurate, but they can still provide valuable insights into sleep patterns. By combining these behavioral tools and strategies, individuals can optimize their sleep quality and experience improved focus, energy, and overall well-being.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman, the best way to improve sleep quality involves a combination of behavioral tools, nutritional strategies, and other techniques. Behavioral tools include:

1. Establishing a consistent morning routine (critical period one)
2. Having an afternoon routine that includes exposure to sunlight
3. Avoiding caffeine in the 8-10 hours leading up to bedtime
4. Not consuming alcohol
5. Cooling down the room temperature
6. Dimming the lights

Additionally, exercise has been shown to improve the quality of slow-wave sleep by increasing slow-wave stability. It's also important to note that individual differences play a role, and what works best may depend on whether someone is interoceptively biased or exteroceptively biased.

Nutrition and supplements can also be considered, but as a secondary approach after establishing good behavioral habits. However, specific recommendations for nutrition and supplements are not mentioned in the provided context.

It's worth noting that commercially available sleep trackers are not perfectly accurate, so while they can provide some insights, they should not be solely relied upon to assess sleep quality.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman's teachings, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some specific tips:

1. **Behavioral Tools:** Establish a consistent morning and afternoon routine, including exposure to sunlight in the afternoon. Avoid caffeine and alcohol close to bedtime (8-10 hours). Cool down the room, dim the lights, and create a relaxing sleep environment.
2. **Nutrition:** While not explicitly mentioned in this context, other sources suggest that a balanced diet rich in nutrients, avoiding heavy meals close to bedtime, and staying hydrated can also support sleep quality.
3. **Specific Strategies for Deep Sleep (Slow-Wave Sleep):** Two well-documented methods to improve slow-wave sleep are:
	* Exercise: Regular exercise can increase slow-wave stability, leading to improved deep sleep.
	* It is unclear what the second method is based on the provided text, but other sources may offer additional insights.

It's essential to note that individual results may vary, and what works for one person might not work for another. Additionally, commercially available sleep trackers are not perfectly accurate, so it's crucial to focus on overall sleep quality rather than relying solely on tracking devices.</div>

--------------------------------

In [42]:
question = "What is the best way to improve sleep quality?"

for i in range(5):
    response = get_rag_response(question, temperature=0.1)
    display(Markdown(f"<div style='font-size: 12pt'>{response}</div>"))
    display(Markdown(f"--------------------------------"))

<div style='font-size: 12pt'>According to Andrew Huberman, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some specific tips he recommends:

1. **Establish a consistent morning and afternoon routine**: This helps regulate your body's internal clock and improves the quality of your sleep.
2. **Get sunlight exposure in the afternoon**: Natural light exposure during the day helps regulate your circadian rhythms, which can lead to better sleep.
3. **Avoid caffeine and alcohol before bedtime**: Both substances can disrupt sleep patterns and reduce the quality of your sleep.
4. **Cool down the room**: A cooler bedroom environment can promote deeper and more restful sleep.
5. **Dim the lights**: Darkness helps signal to your body that it's time to sleep, making it easier to fall asleep and stay asleep.
6. **Exercise regularly**: Exercise can help improve the quality of slow-wave sleep, also known as deep sleep. However, the timing and intensity of exercise may need to be adjusted based on individual factors, such as being interoceptively or exteroceptively biased.
7. **Consider nutritional interventions**: While not specified in this particular text, Andrew Huberman mentions looking into nutrition as a potential tool for improving sleep quality.

It's worth noting that Andrew Huberman emphasizes the importance of addressing sleep issues through behavioral changes before considering other interventions. Additionally, he highlights the limitations of commercially available sleep trackers, suggesting that they may not always provide accurate readings.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Some specific recommendations include:

1. Establishing a consistent morning and afternoon routine.
2. Exposing yourself to sunlight in the afternoon.
3. Avoiding caffeine for 8-10 hours before bedtime.
4. Not consuming alcohol.
5. Cooling down the room temperature.
6. Dimming the lights.

Additionally, exercise has been shown to improve slow-wave sleep quality by increasing slow-wave stability. It's also important to note that individual results may vary depending on whether someone is interoceptively biased (more focused on internal bodily sensations) or exteroceptively biased (more focused on external stimuli).

It's worth noting that while sleep trackers can provide some insights, they are not perfectly accurate, especially commercially available ones. Therefore, it's essential to focus on developing healthy sleep habits rather than relying solely on technology.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman's teachings, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some specific tips he recommends:

1. **Establish a consistent morning and afternoon routine**: This helps regulate your body's internal clock and improves the quality of your sleep.
2. **Get sunlight exposure in the afternoon**: Natural light exposure during the day helps regulate your circadian rhythms, leading to better sleep.
3. **Avoid caffeine and alcohol before bedtime**: Both substances can disrupt sleep patterns and reduce sleep quality.
4. **Cool down the room**: A cooler bedroom environment promotes deeper sleep.
5. **Dim the lights**: Darkness signals to your body that it's time to sleep, helping you fall asleep faster and sleep more soundly.
6. **Exercise regularly**: Exercise can improve slow-wave sleep stability, which is essential for deep, restorative sleep.
7. **Consider nutritional interventions**: While not specified in this particular text, Andrew Huberman may recommend certain nutrients or supplements to support sleep quality.

It's worth noting that Andrew Huberman emphasizes the importance of addressing sleep issues through behavioral changes before considering other interventions. By incorporating these habits into your daily routine, you can significantly improve the quality of your sleep.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Some specific recommendations include:

1. Establishing a consistent morning and afternoon routine, including exposure to sunlight in the afternoon.
2. Avoiding caffeine and alcohol in the hours leading up to bedtime.
3. Cooling down the room and dimming the lights to create a sleep-conducive environment.
4. Engaging in exercise, which has been shown to improve slow-wave sleep stability (although the timing and type of exercise are not specified).
5. Considering nutritional approaches, although this is mentioned as a secondary step after implementing behavioral changes.

It's also important to note that commercial sleep trackers may not be perfectly accurate, but can still provide useful insights into sleep patterns. Additionally, getting enough sleep and maintaining good sleep quality can have a significant impact on overall well-being, energy levels, and mood.</div>

--------------------------------

<div style='font-size: 12pt'>According to Andrew Huberman's teachings, the best way to improve sleep quality involves a combination of behavioral tools, nutrition, and other strategies. Here are some specific tips:

1. **Establish a consistent morning and afternoon routine**: This helps regulate your body's internal clock and improves the quality of your sleep.
2. **Get sunlight exposure in the afternoon**: Natural light exposure during the day helps regulate your circadian rhythms, leading to better sleep.
3. **Avoid caffeine and alcohol before bedtime**: Both substances can disrupt sleep patterns and reduce sleep quality.
4. **Cool down the room temperature**: A cooler room promotes deeper sleep and improves sleep quality.
5. **Dim the lights**: Darkness signals the body to produce melatonin, which regulates sleep-wake cycles.
6. **Exercise regularly**: Exercise can help improve slow-wave sleep stability, but the timing and type of exercise may vary depending on individual factors (e.g., being interoceptively or exteroceptively biased).
7. **Consider nutritional interventions**: While not explicitly mentioned in this specific text, Andrew Huberman suggests considering nutrition as a second step after implementing behavioral changes.

It's essential to note that individual results may vary, and what works for one person might not work for another. Additionally, sleep trackers can provide insights into sleep patterns, but their accuracy is limited, and they should not be solely relied upon for assessing sleep quality.</div>

--------------------------------

### ***similarity seach breakdown***

***Who is Andrew Huberman?***

#### ***cosine***

In [61]:
query = "Who is Andrew Huberman?"
top_docs = get_top_docs_cosine(vector_store, query)
for doc in top_docs:
    print(doc.page_content.replace("\n", " "))
    print("--------------------------------")


- Welcome to the Huberman Lab Podcast where we discuss science and science-based tools for everyday life. Welcome, I'm Andrew Huberman, and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. That's what I do for my day job, but over the last few years I've become very active in teaching neuroscience and neuroscience-related themes on social media. And I've also been fortunate enough to be hosted as a guest on various people's podcasts to discuss neuroscience and
--------------------------------
- [Andrew Huberman] Welcome to the Huberman Lab Podcast, where we discuss science and science-based tools for everyday life. - My name is Andrew Huberman, and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. This podcast is separate from my teaching and research roles at Stanford. It is however part of my desire and effort to bring zero-cost to consumer information about science and science-related tools to the general public. In ke

#### ***mmr***

In [62]:
top_docs = get_top_docs_mmr(vector_store, query, k=6, fetch_k=100, lambda_mult=0.9)
for doc in top_docs:
    print(doc.page_content.replace("\n", " "))
    print("--------------------------------")

- Welcome to the Huberman Lab Podcast where we discuss science and science-based tools for everyday life. Welcome, I'm Andrew Huberman, and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. That's what I do for my day job, but over the last few years I've become very active in teaching neuroscience and neuroscience-related themes on social media. And I've also been fortunate enough to be hosted as a guest on various people's podcasts to discuss neuroscience and
--------------------------------
- [Andrew Huberman] Welcome to the Huberman Lab Podcast, where we discuss science and science-based tools for everyday life. - My name is Andrew Huberman, and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. This podcast is separate from my teaching and research roles at Stanford. It is however part of my desire and effort to bring zero-cost to consumer information about science and science-related tools to the general public. In ke

***What is the best way to improve sleep quality?***

#### ***cosine***

In [65]:
query = "What is the best way to improve sleep quality?"
top_docs = get_top_docs_cosine(vector_store, query)
for doc in top_docs:
    print(doc.page_content.replace("\n", " "))
    print("--------------------------------")

early part of the night, more of that time is going to be occupied by slow-wave sleep. The two ways to improve slow-wave sleep that are well documented in the literature, and here we can point to some really nice papers that I'll reference in case you want to read further, is the first one is entitled, Exercise Improves the Quality of Slow-Wave Sleep by Increasing Slow-Wave Stability. Slow-wave Stability has to do with, as researchers call it, the amount of power present
--------------------------------
again, should be dictated by whether or not you tend to be interoceptively biased or exteroceptively biased. But if you want to get better at sleeping, you want to get better at falling asleep, and you want to replace sleep that you've lost, I put that in quotes so that my colleagues like Matthew Walker don't come after me with... What would you come after me with, Matt? Probably with an alarm clock and, I don't know, blankets and a pillow or something of that sort. In all seriousness,


#### ***mmr***

In [66]:
top_docs = get_top_docs_mmr(vector_store, query, k=6, fetch_k=100, lambda_mult=0.9)
for doc in top_docs:
    print(doc.page_content.replace("\n", " "))
    print("--------------------------------")

early part of the night, more of that time is going to be occupied by slow-wave sleep. The two ways to improve slow-wave sleep that are well documented in the literature, and here we can point to some really nice papers that I'll reference in case you want to read further, is the first one is entitled, Exercise Improves the Quality of Slow-Wave Sleep by Increasing Slow-Wave Stability. Slow-wave Stability has to do with, as researchers call it, the amount of power present
--------------------------------
start to implement some, or ideally, all of these tools, the quality of your sleep will increase tremendously. And of course, in doing so, the quality of your daytime alertness and your ability to focus will improve tremendously. Again, sleep is the absolute foundation of your mental health, your physical health, and your performance in all endeavors. So if there's one area of your life to really focus on and try and optimize, if your goal is to be happier and more productive
----------